# Код для соревнования https://www.kaggle.com/competitions/platesv2/overview


In [4]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
X_train = df_train.text
y_train = df_train.target

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [71]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/2", trainable = True)

In [73]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [74]:
l = tf.keras.layers.Dropout(0.2, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output2")(l)

In [75]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [76]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=opt,
 loss='binary_crossentropy',
 metrics=METRICS)

In [77]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
238/238 [==============================] - 275s 1s/step - loss: 0.4863 - accuracy: 0.7839 - precision: 0.7841 - recall: 0.6860
Epoch 2/5
238/238 [==============================] - 259s 1s/step - loss: 0.3870 - accuracy: 0.8392 - precision: 0.8600 - recall: 0.7475
Epoch 3/5
238/238 [==============================] - 260s 1s/step - loss: 0.3373 - accuracy: 0.8634 - precision: 0.8829 - recall: 0.7863
Epoch 4/5
238/238 [==============================] - 259s 1s/step - loss: 0.3082 - accuracy: 0.8761 - precision: 0.8930 - recall: 0.8086
Epoch 5/5
238/238 [==============================] - 260s 1s/step - loss: 0.2605 - accuracy: 0.9015 - precision: 0.9167 - recall: 0.8478


In [78]:
y_predicted = model.predict(df_test.text)
y_predicted = y_predicted.flatten()

102/102 [==============================] - 33s 325ms/step


In [79]:
pred = (y_predicted > 0.5).astype(int).reshape(1,-1)[0]
df = pd.DataFrame({'id':df_test.id,'target':pred})
df.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1


In [80]:
df.to_csv('result.csv', index = False)